In [2]:
!java -version
!javac -version

java version "1.8.0_341"
Java(TM) SE Runtime Environment (build 1.8.0_341-b10)
Java HotSpot(TM) Client VM (build 25.341-b10, mixed mode)
'javac' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!python -m pip install tabula-py
!pip install --ignore-installed xlsxwriter

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


  Using cached XlsxWriter-3.0.3-py3-none-any.whl (149 kB)


In [6]:
def guessed_table_to_df(guessed,uni):
    df_temp = guessed.rename(columns={0:'Region',1:'Rank in Region', 2: 'Number of Exams', 3: 'GPA', 4: 'Mean Score Kanoon', 5:'City', 
                          6: 'Student Name', 7: 'Row YM', 8:'Region', 9:'Rank in Region', 10: 'Number of Exams', 11: 'GPA', 
                          12: 'Mean Score Kanoon', 13:'City', 14: 'Student Name', 15: 'Row YM'})
    left = df_temp.iloc[:, :8]
    right = df_temp.iloc[:, 8:]
    table = pd.concat([right,left.rename(columns={0:8})], ignore_index=True)
    table['Uni Maj'] = uni
    return table

In [7]:
def fun_name(df):
    df['Student Name'] = df['Student Name'].str.replace('روح االله','روح‌الله')
    df['Student Name'] = df['Student Name'].str.replace(' ااالله','‌االله')
    df['Student Name'] = df['Student Name'].str.replace(' االله','‌الله')
    df['Student Name'] = df['Student Name'].str.replace('پانته ا','پانته‌آ')
    if Year in [1386, 1387, 1389, 1390]:
        df['Student Name'] = df['Student Name'].str.replace(r"\).*\(","")
        df['Student Name'] = df['Student Name'].str.replace(r"\).*","")
        df['Student Name'] = df['Student Name'].str.replace(r"\(","")
        df['Student Name'] = df['Student Name'].str.strip()
        df['First Name'] = df['Student Name'].str.rsplit(' ',1).str[-1]
        df['Last Name'] = df['Student Name'].str.rsplit(' ',1).str[0]
        df['First Name'] = df['First Name'].str.strip()
        df['Last Name'] = df['Last Name'].str.strip() 
    return

In [8]:
def clean_columns(df):
    df['Region'] = df['Region'].replace('ساير',4)
    for column in ['Student Name','City','Uni Maj']:
        try:
            df[column] = df[column].str.replace("ي","ی")
            df[column] = df[column].str.replace("ئ","ی")
            df[column] = df[column].str.replace("ك","ک")
        except:
            print('Something wrong with ' + column)
    for column in ['Mean Score Kanoon', 'Number of Exams', 'Region', 'Rank in Region', 'Row YM', 'GPA']:
        try:
            df[column] = pd.to_numeric(df[column], errors= 'raise')
        except Exception as e: print(e)
        df[column] = pd.to_numeric(df[column], errors= 'coerce')
    return

In [9]:
def fun_uni_maj(df):
    if Year in [1386, 1387, 1389, 1390]:
        df['Uni Maj'] = df['Uni Maj'].str.replace(")","$")
        df['Uni Maj'] = df['Uni Maj'].str.replace("(",")")
        df['Uni Maj'] = df['Uni Maj'].str.replace("$","(")
        df['Uni Maj'] = df['Uni Maj'].str.replace(r"--\)",")--")
        df['Major'] = df['Uni Maj'].str.split("--").str[0]
        df['University'] = df['Uni Maj'].str.split("--").str[1]
        df['University'] = df['University'].str.strip()
        df['Major'] = df['Major'].str.strip()
        df['Subfield'] = df['Major'].str.split("(").str[1]   
        df['Major'] = df['Major'].str.split("(").str[0]
    try:
        df['Subfield'] = df['Subfield'].str.replace(")","")
    except:
        pass
    return

In [10]:
def repeated_df_remover(dfs_guess):
    for i in range(0, len(dfs_guess)): # This is to remove the first couple of lines where the title is also included in table.
        if len(dfs_guess[i].columns) < 16:
            print("There is a problem with guessed table:  ", i+1, " in page:  ", pp)
            del dfs_guess[i]
            continue
        while dfs_guess[i].iloc[0][0] == np.nan or dfs_guess[i].iloc[0].str.contains("چارک|ثبت|منطقه|سهميه|رديف|--|فارغ التحصيل|اطلاعات|تعداد").any():
            dfs_guess[i] = dfs_guess[i].drop(dfs_guess[i].index[0])
    kk = len(dfs_guess)
    if kk > 1:          # To Remove any duplicate tables. Those who have a single column get duplicated somehow.
        for i in range(kk-1,0,-1):
            x = dfs_guess[i].iloc[0] 
            y = dfs_guess[i-1].iloc[0]
#             print(i, i-1 , x.equals(y))
#             print(x)
#             print(y)
            if x.equals(y) or x[14] == y[14]:
                del dfs_guess[i]
    return dfs_guess

In [13]:
def content_to_df(df, content):
    try:
        Unis.clear()
    except:
        pass
    flag = 0 #This flag is used to distinguish the name of the University with the extra names at the end of each table.
    extras = ""
    
    for i in range(0,len(content)-1):
        try:
            x = content.iloc[i]
            if x.str.contains("پيوستگي").any():
#                 print("peyvastegi", i)
                flag = 1
                continue
            if x.str.contains("چارک|ثبت|منطقه|سهميه|ارائه شده|فارغ التحصيل|A|دفترچه|B|نوع رشته").any():
                continue
            if len(x)-x.isna().sum() == 1:
                y = [a for a in x if pd.isnull(a) == False and a != 'nan'][0]
#                 print(y)
                if "--" in y and ("موسسه" in y or "دانشگاه" in y or "دانشكده" in y or "مجتمع" in y \
                                  or "غيرانتفاعي" in y or "مركز آموزش عالي" in y or "علوم پايه" in y):
                    if extras != "":
#                         print("Appending extras")
                        for jj in extras.split("--"):
                            df = df.append({'Student Name': jj, 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
                        extras = ""
                    Unis.append(y)
                    flag = 0
                    continue
                elif flag == 1:
                    extras = extras + " " + y
                    continue
            if Year == 1386:
                df = single_column_1386(df, x)           
            elif Year == 1387:
                df = single_column_1387(df, x)           
            elif Year == 1389:
                df = single_column_1389(df, x)  
            elif Year == 1390:
                df = single_column_1389(df, x)        
            elif Year == 1391:
                df = single_column_1391(df, x)
                            
            
        except Exception as e:
            print("\t There is a problem with line: ", i, " of content \t", e)
    
    if extras != "":
#         print("Appending extras")
        for jj in extras.split("--"):
            df = df.append({'Student Name': jj, 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
        extras = ""
    
    return df, Unis

# Main Code

First, it reads the file, then loops over all pages. Starts with extracting the Universities included in a given page and then attaches the guessed tables to it.

In [14]:
import tabula
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.DataFrame(columns=['Student Name', 'City', 'Mean Score Kanoon', 'GPA', 'Number of Exams', 
                           'Region', 'Rank in Region', 'Uni Maj', 'Page YM', 'Row YM'])
errs = []
problematic = []
missing_row = []

main_path = "G:\My Drive\ECON\Research\Archive\konkoor\Iranian Data\Ghalamchi"
Year = 1388
Track = 'Riazi'
file_path = "G:\My Drive\ECON\Research\Archive\konkoor\Iranian Data\Ghalamchi\\1388\\1388 Mohandesi.pdf"

for pp in range(10,11):
    print('Working on page: \t', pp)
    dfs_guess = tabula.read_pdf(file_path, stream=True, 
                                pandas_options={'header': None}, pages = pp)
#     print('\t Number of tables with guess: \t', len(dfs_guess))

    if pp in [69]:
        dfs_guess[0].to_excel("Output/" + "prob69 "+ str(Year)+ " " + str(Track) +".xlsx", index=False)
        continue
        
    
    dfs_guess = repeated_df_remover(dfs_guess)
    
    content = tabula.read_pdf(file_path, stream=True, guess=False, 
                              pandas_options={'header': None}, pages = pp)

    df, Unis = content_to_df(df, content[0])
    
    
    for i in range(0,len(Unis)):
        table = guessed_table_to_df(dfs_guess[i],Unis[i])
        table['Page YM'] = pp
#         print(table.head())

        if 'Row YM' not in table.columns:            
            print("\t There is something wrong with page:  ", pp)
            continue
            
        if table['Row YM'].isna().sum() > 3:
            print("\t There is something wrong with page:  ", pp, "\t Table: \t", i+1)
#             print('\t Number of tables with guess: \t', len(dfs_guess))
#             print('\t Number of tables without guess: ', len(content))
#             print('\t Number of rows without guess: ',len(content[0]))
#             print('\t Number of Columns without guess: ',len(content[0].columns))
            errs.insert(0,[pp, i])
            problematic.append(table)
            continue
            
        elif table['Row YM'].isna().sum() > 0:
            print("\t There are some rows missing in page:  ", pp)
            missing_row.append(pp)
            df = df.append(table)
        else:
            df = df.append(table)
            
    df['Year'] = Year
    df['Track'] = Track

print("Done!")

Working on page: 	 10


Got stderr: Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:39 AM org.apache.pdfb

Got stderr: Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Sep 20, 2022 11:57:40 AM org.apache.pdfb

NameError: name 'Unis' is not defined

In [790]:
df2 = df.copy()
clean_columns(df2)
fun_name(df2)
fun_uni_maj(df2)
df2 = df2[['Year', 'First Name', 'Last Name', 'City', 'Mean Score Kanoon', 'GPA', 'Number of Exams', 
         'Region', 'Rank in Region', 'Uni Maj', 'University', 'Major', 'Subfield', 'Track', 'Page YM', 'Row YM']]
df2 = df2.reset_index(drop=True)
df2.sort_values(by=['Year', 'Page YM', 'Row YM', 'Uni Maj', 'Region', 'Rank in Region'])

# df.to_csv("Output/"+ str(Year)+ str(Track) +".csv", index=False, encoding="utf-8")
df2 = df2.sort_values(by=['Year', 'Page YM', 'Uni Maj', 'Region', 'Rank in Region'])
df2.to_excel(main_path + "/Pdf read/Output/"+ str(Year)+ " " + str(Track) +".xlsx", index=False) 
print("Done!")

Done!


# Checks

Here are the codes for checking the data 

In [769]:
Unis

['پزشكي--دانشگاه علوم پزشكي تبريز)روزانه (']

In [4]:
df

,Student Name,City,Mean Score Kanoon,GPA,Number of Exams,Region,Rank in Region,Uni Maj,Page YM,Row YM


In [15]:
content[0]

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",=/",...,"-$),",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,:,NaN,J,NaN,NaN,NaN,NaN,NaN,6839 :,NaN,...,NaN,NaN,NaN,NaN,= 130 = 78,$ #,NaN,7.0,NaN,$%
2,215 : 3,NaN,'(),NaN,!,NaN,NaN,NaN,898 : 2,'(),...,NaN,NaN,NaN,NaN,NaN,A++:,NaN,NaN,NaN,5
3,NaN,NaN,NaN,$%,NaN,NaN,"!"" !",NaN,NaN,NaN,...,"!"" !",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"!+,-",NaN,NaN,NaN,#$%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,*,NaN,'(),&,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2 783,NaN,NaN,11,19,NaN,7188,NaN,NaN,NaN,...,7834,NaN,NaN,&',NaN,;- q >,NaN,NaN,NaN,1
7,1 812,NaN,NaN,17,18,NaN,6896,NaN,&',"""",...,7718,NaN,NaN,NaN,NaN,"""",NaN,NaN,NaN,2
8,2 842,NaN,NaN,19,17,NaN,7269,NaN,NaN,"(V)"" +",...,6545,NaN,[,/,;,"! ""$ *",NaN,NaN,NaN,3
9,2 850,NaN,NaN,18,19,NaN,6623,NaN,""")*",NaN,...,6704,NaN,Y,NaN,NaN,*,NaN,NaN,NaN,4


In [794]:
dfs_guess[0]

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,NaN,NaN,NaN,NaN,NaN,پزشكي--دانشگاه علوم پزشكي تبريز)روزانه (,NaN,NaN,NaN,NaN,NaN
1,NaN,رتبه در,تعداد,NaN,ميانگين تراز,محل ثبت نام,تعداد رتبه در,NaN,ميانگين تراز,محل ثبت نام,NaN,NaN
2,سهميه,NaN,NaN,معدل,NaN,NaN,سهميه رديف نام خانوادگي و نام,معدل,NaN,NaN,نام خانوادگي و نام,رديف
3,NaN,منطقه *,آزمون,NaN,در كانون,در كانون,آزمون منطقه *,NaN,در كانون,در كانون,NaN,NaN
4,2,438,9,19,7289,اردبيل,عيني بايندوري مهسا53 1 5 19,20,8105,تبريز,كلاهدوزان كسري,1
5,2,439,18,NaN,6985,اروميه,معمارزاده ارمين54 1 24 16,19,7675,تبريز,راد بابك,2
6,1,440,5,19,7244,تهران,قره باغي ميلاد)ف(55 1 37 19,19,7724,تبريز,حامدفر هادي,3
7,2,445,19,19,7104,مياندوآب,حيدري مياندواب مهسا56 3 48 18,19,6959,هريس سراب,فخيمي هريس طاهره,4
8,2,480,19,19,7112,مرند,محمدي النجقي علي57 1 50 19,19,7906,تبريز,چاوشي سيدمحمدرضا,5
9,1,484,19,18,6725,تبريز,نجاريان فريما58 1 80 17,19,7675,تبريز,صاحب كرم علمداري نگين,6


In [115]:
df.to_excel(main_path + "/Pdf read/Output/" + "temp "+ str(Year)+ " " + str(Track) +".xlsx", index=False) 

# Errors

Here are the codes to deal with the problematic pages

In [507]:
problematic[0]

IndexError: list index out of range

In [671]:
len(content[0].columns)

16

In [796]:
df_prob = tabula.read_pdf(file_path, area=[140,72,720,285], stream=True, 
                                pandas_options={'header': None}, pages = 69)
df_prob[0]

,0,1,2,3,4,5,6
0,39,18,NaN,6985,اروميه,معمارزاده ارمين,54
1,40,5,19.0,7244,تهران,قره باغي ميلاد)ف(,55
2,45,19,19.0,7104,مياندوآب,حيدري مياندواب مهسا,56
3,80,19,19.0,7112,مرند,محمدي النجقي علي,57
4,84,19,18.0,6725,تبريز,نجاريان فريما,58
5,16,17,19.0,7049,تبريز,امامي زاده كلوجه ساناز)ف(,59
6,27,17,NaN,6924,تبريز,اسماعيلي سپيده,60
7,36,11,19.0,6363,تبريز,پيرزاده سپهر,61
8,39,19,19.0,7116,خوي,رحيم اوغلي مهديه,62
9,64,19,18.0,6987,اروميه,صفرعليزاده علي)ف(,63


In [799]:
df_prob = tabula.read_pdf(file_path, area=[140,72,720,285], stream=True, 
                                pandas_options={'header': None}, pages = 69)
# table_prob = guessed_table_to_df(df_prob[0],"Problematic -- Problematic")

# clean_columns(table_prob)
fun_name(table_prob)
fun_uni_maj(table_prob)
#table_prob = table_prob[['Year', 'First Name', 'Last Name', 'City', 'Mean Score Kanoon', 'Number of Exams', 
#         'Region', 'Rank in Region', 'Uni Maj', 'University', 'Major', 'Subfield', 'Track', 'Page YM', 'Row YM']]
table_prob = table_prob.reset_index(drop=True)
#table_prob.sort_values(by=['Year', 'Page YM', 'Row YM', 'Uni Maj', 'Region', 'Rank in Region'])

table_prob.to_excel("Output/" + "prob1 "+ str(Year)+ " " + str(Track) +".xlsx", index=False) 

AttributeError: Can only use .str accessor with string values!

In [23]:
dfs_guess_prob = tabula.read_pdf(file_path, stream=True, 
                                pandas_options={'header': None}, pages = 125)

In [31]:
dfs_guess_prob[1]

,0,1,2,3,4,5,6,7,8,9,10,11
4,2,12828,16,18,5570,بيرجند,هاشمي حسين)ف(10 2 5423 14,19,5181,بيرجند,صحرائي حاجي اباد مريم,1
5,2,13024,15,17,4831,بيرجند,قوسي سجاد11 3 6270 13,NaN,5283,سرخس,كريمي نويد)ف(,2
6,2,14159,9,NaN,5991,بيرجند,طوقي سعيد)ف(12 1 6820 16,NaN,5346,مشهد,كامل خدابنده شيرين)ف(,3
7,2,14159,17,18,5163,شيراز,وزيرزاده مينا13 3 6872 22,19,5406,بيرجند,نوري گزيك الناز,4
8,2,14283,18,18,5606,بيرجند,نخعي زينلي الهام)ف(14 1 6969 20,18,5384,مشهد,عرب زهره)ف(,5
9,2,14296,22,18,5000,بيرجند,چهكندي نژاد زهرا15 1 7779 24,18,5100,مشهد,وكيلي فدافن سحر,6
10,2,14518,14,19,5515,كاشمر,مصطفائي محدثه16 1 8299 16,NaN,5221,اصفهان,زارع اندلاني سعيد,7
11,2,14935,24,18,5313,بيرجند,مودي افسانه17 1 8691 15,19,5217,مشهد,شاهپوري ساناز,8
12,2,15481,18,17,5064,بيرجند,محمودي نجمه)ف(18 3 9979 25,19,5381,تايباد,بابائي حديثه,9


In [512]:
Unis[1]

IndexError: list index out of range

# Different Years Content

Below are the codes for the content of different years

In [411]:
def single_column_1391(df, x):
    if len(x)-x.isna().sum() == 3:
        print(x)
        if len(x) == 9:
            lettered = "".join(m for m in x[6] if not m.isdigit())
            numbered = x[6].replace(lettered.strip(),"")
            y = numbered.split()
            if len(y) < 5:
                y.append(y[3])
                y[3] = np.nan
            df = df.append({'Row YM': x[8], 'Student Name': x[7], 'City': lettered.strip(), 
                            'Mean Score Kanoon': y[4], 'Number of Exams': y[2], 'Region': y[0], 'Rank in Region': y[1], 
                            'GPA': y[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)                
        elif len(x) == 8:
            lettered = "".join(m for m in x[5] if not m.isdigit())
            numbered = x[5].replace(lettered.strip(),"")
            y = numbered.split()
            if len(y) < 5:
                y.append(y[3])
                y[3] = np.nan
            df = df.append({'Row YM': x[7], 'Student Name': x[6], 'City': lettered.strip(), 
                            'Mean Score Kanoon': y[4], 'Number of Exams': y[2], 'Region': y[0], 'Rank in Region': y[1], 
                            'GPA': y[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)                
        elif len(x) == 10:
            lettered = "".join(m for m in x[5] if not m.isdigit())
            numbered = x[5].replace(lettered.strip(),"")
            y = numbered.split()
            if len(y) < 5:
                y.append(y[3])
                y[3] = np.nan
            df = df.append({'Row YM': x[7], 'Student Name': x[6], 'City': lettered.strip(), 
                            'Mean Score Kanoon': y[4], 'Number of Exams': y[2], 'Region': y[0], 'Rank in Region': y[1], 
                            'GPA': y[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)

In [733]:
def single_column_1389(df, x):
    if len(x)-x.isna().sum() == 1:
        if len(content[0].columns) == 1:
            return df
        
        y = [a for a in x if pd.isnull(a) == False and a != 'nan'][0]
#         print("single_column")
        lettered = "".join(m for m in y if not m.isdigit())
#         print("lettered", lettered)
        numbered = y.replace(lettered.split("  ")[0],"").replace(lettered.split("  ")[-1],"")
#         print("numbered", numbered)
        z = numbered.split()
#         print(z)
        if len(z) == 3:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 4:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
        elif len(z) == 5:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': z[5], 'Student Name': lettered.split("  ")[0], 'City': lettered.split("  ")[-1], 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    elif len(x)-x.isna().sum() == 3:
        if len(content[0].columns) == 3 or not pd.isnull(x[0]):
            return df
        if len(content[0].columns) == 8:
            stud_name = x.iat[-2]
            row_ym = x.iat[-1]
            y = x.iat[-3]
        elif len(content[0].columns) == 9:
            row_ym = x.iat[-1]
            stud_name = x.iat[-2]
            y = x.iat[-3]
        elif len(content[0].columns) == 10:
            stud_name = x.iat[-3]
            row_ym = x.iat[-1]
            y = x.iat[-4]            
#         print(y)
        city = "".join(m for m in y if not m.isdigit()).strip()
#         print("city", city)
        numbered = y.replace(city,"")
#         print("numbered", numbered)
        z = numbered.split()
#         print(z)
        if len(z) == 2:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 3:
            z.insert(1, np.nan)
            z.insert(3, np.nan)
        elif len(z) == 4:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': row_ym, 'Student Name': stud_name, 'City': city, 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
        
    elif len(x)-x.isna().sum() == 4:
        if len(content[0].columns) == 4 or not pd.isnull(x[0]):
            return df
        if len(content[0].columns) == 9:
            stud_name = x.iat[-2]
            row_ym = x.iat[-1]
            y = x.iat[-3]
        elif len(content[0].columns) == 10:
            row_ym = x.iat[-1]
            stud_name = x.iat[-2]
            city = x.iat[-3]
            y = x.iat[-4]     
        elif len(content[0].columns) == 11:
            row_ym = x.iat[-1]
            stud_name = x.iat[-2]
            city = x.iat[-3]
            y = x.iat[-5]          
#         print(y)
        z = y.split()
#         print(z)
        if len(z) == 2:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 3:
            z.insert(1, np.nan)
            z.insert(3, np.nan)
        elif len(z) == 4:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': row_ym, 'Student Name': stud_name, 'City': city, 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    return df

In [555]:
def single_column_1387(df, x):
    if len(x)-x.isna().sum() == 1:
        if len(content[0].columns) == 1:
            return df
        
        y = [a for a in x if pd.isnull(a) == False and a != 'nan'][0]
#         print("single_column")
        lettered = "".join(m for m in y if not m.isdigit())
#         print("lettered", lettered)
        numbered = y.replace(lettered.split("  ")[0],"").replace(lettered.split("  ")[-1],"")
#         print("numbered", numbered)
        z = numbered.split()
#         print(z)
        if len(z) == 3:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 4:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
        elif len(z) == 5:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': z[5], 'Student Name': lettered.split("  ")[0], 'City': lettered.split("  ")[-1], 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    elif len(x)-x.isna().sum() == 2:
        if len(content[0].columns) == 2 or not pd.isnull(x[0]):
            return df
        stud_name = "".join(m for m in x.iat[-1] if not m.isdigit()).strip()
        row_ym = "".join(m for m in x.iat[-1] if m.isdigit()).strip()
#         print(x)
        y = x.iat[-2]
#         print(y)
        lettered = "".join(m for m in y if not m.isdigit()).strip()
#         print("lettered", lettered)
        numbered = y.replace(lettered,"")
#         print("numbered", numbered)
        z = numbered.split()
#         print(z)
        if len(z) == 2:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 3:
            z.insert(1, np.nan)
            z.insert(3, np.nan)
        elif len(z) == 4:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': row_ym, 'Student Name': stud_name, 'City': lettered, 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    elif len(x)-x.isna().sum() == 3:
        if len(content[0].columns) == 3 or not pd.isnull(x[0]):
            return df
        stud_name = "".join(m for m in x.iat[-1] if not m.isdigit()).strip()
        row_ym = "".join(m for m in x.iat[-1] if m.isdigit()).strip()
        city = x.iat[-2]
#         print(x)
        y = x.iat[-3]
#         print(y)
        z = y.split()
#         print(z)
        if len(z) == 2:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan) 
        elif len(z) == 3:
            z.insert(1, np.nan)
            z.insert(3, np.nan)
        elif len(z) == 4:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': row_ym, 'Student Name': stud_name, 'City': city, 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    return df

In [395]:
def single_column_1386(df, x):
    if len(x)-x.isna().sum() == 1:
        y = [a for a in x if pd.isnull(a) == False and a != 'nan'][0]
#         print("single_column")
        lettered = "".join(m for m in y if not m.isdigit())
#         print("lettered", lettered)
        numbered = y.replace(lettered.split("  ")[0],"").replace(lettered.split("  ")[-1],"")
#         print("numbered", numbered)
        z = numbered.split()
#         print(z)
        if len(z) == 3:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
            z.insert(3, np.nan)
        elif len(z) == 4:
            z.insert(0, np.nan)
            z.insert(1, np.nan)
        elif len(z) == 5:
            z.insert(3, np.nan)
        
        df = df.append({'Row YM': z[5], 'Student Name': lettered.split("  ")[0], 'City': lettered.split("  ")[-1], 
                        'Mean Score Kanoon': z[4], 'Number of Exams': z[2], 'Region': z[0], 'Rank in Region': z[1], 
                        'GPA': z[3], 'Uni Maj': Unis[-1], 'Page YM': pp}, ignore_index=True)
    
    return df